## In this juptyer notebook, we predict kcat values for a metabolic network that will be used to predict proteome allocations. The created csv file has to be stored in the a folder called "PredcitedKcat343species" downloaded from Li et al.

In [16]:
import pandas as pd
import numpy as np
import os
from os.path import join
import cobra.io
import cobra
from pubchempy import *
import pickle
import xgboost as xgb

from Bio import SeqIO
from rdkit import Chem
from rdkit.Chem import AllChem

import warnings
warnings.filterwarnings('ignore')

strain = "Kluyveromyces_lactis"

In [2]:
model = cobra.io.load_matlab_model(join("..", "..", "data", "DLKcat", "ssGEMs", strain + ".mat"))

df_kcat = pd.read_csv(join("..", "..", "data", "DLKcat", "PredcitedKcat343species",
                 strain + "_PredictionResults.txt"), sep = "\t")
df_kcat

,# rxnID,MNXID,met_model,met_standard_name,Smiles,genes,refID,Kcat value (substrate first)
0,r_0001_1,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_1887;Kluyveromyces_la...,chebi:16004;chebi:15991,"0.4359,21.0893;"
1,r_0001_2,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,2.9483;"
2,r_0001_3,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,3.3644;"
3,r_0001_4,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,2.0915;"
4,r_0002,MNXM731974;MNXM731834,ferricytochrome c;(R)-lactate,ferricytochrome c;(R)-lactate,;C[C@@H](O)C([O-])=O,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:15991;chebi:16004,";21.0893,2.9483"
...,...,...,...,...,...,...,...,...
4639,p_5228_rvs,MNXM729613,D-fructose,D-fructose,OC[C@H]1OC(O)(CO)[C@@H](O)[C@@H]1O,Kluyveromyces_lactis@Seq_1175,chebi:37721,0.4837
4640,r_0486_1_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_2416,chebi:57540;chebi:43474;chebi:17138,12.1500;28.4597;54.7794
4641,r_0486_1_rvs,MNXM10;MNXM733151,"NADH;1,3-bisphospho-D-glycerate",NADH;(2R)-3-phospho-glyceroyl phosphate,NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP([O-])(=O)O...,Kluyveromyces_lactis@Seq_2416,chebi:57945;chebi:57604,31.1839;27.5817
4642,r_0486_2_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_3619,chebi:57540;chebi:43474;chebi:17138,12.2171;17.5832;35.2701


## 1.Getting CHEBI IDs and SMILES for all metabolites:

### (a) CHEBI IDs:

In [4]:
df_yeast = pd.read_csv(join("..", "..", "data", "DLKcat", "MNX_met_yeast8.3.4.txt"),
           sep = "\t", header = None)
df_yeast.head()

,0,1,2,3,4,5,6
0,MNXM01,PMF,NaN,H,1.007940,1.0,mnx:PMF;MNXM01
1,MNXM1,H(+),s_0793;s_0794;s_0795;s_0796;s_0797;s_0798;s_07...,H,1.007940,1.0,chebi:15378;MNXM1;biggM:M_h;biggM:h;chebi:1074...
2,MNXM10,NADH,s_1203;s_1204;s_1205;s_1206;s_2818;s_3753,C21H27N7O14P2,663.425382,-2.0,chebi:57945;biggM:M_nadh;biggM:nadh;chebi:1339...
3,MNXM100,(2E)-geranyl diphosphate,s_0745,C10H17O7P2,311.185302,-3.0,chebi:58057;biggM:M_grdp;biggM:grdp;chebi:1429...
4,MNXM1018,D-arabinitol,s_4181,C5H12O5,152.145780,0.0,chebi:18333;biggM:M_abt__D;biggM:abt__D;chebi:...


In [5]:
def find_chebi(xrefs):
    for ref in xrefs:
        if "chebi" in ref:
            return(ref)
    return(None)

def remove_dups (dct):
    reversed_dct = {}
    for key, val in dct.items():
        new_key = tuple(val["dst"]) + tuple(val["src"]) + (tuple(val["alias"]) if "alias" in val else (None,) ) 
        reversed_dct[new_key] = key
    result_dct = {}
    for key, val in reversed_dct.items():
        result_dct[val] = dct[val]
    return result_dct

In [6]:
df_mets = pd.DataFrame(columns = ["ID", "CHEBI"])


for ind in df_yeast.index:
    IDs = df_yeast[2][ind]
    xrefs = df_yeast[6][ind]
    if not pd.isnull(IDs) and not pd.isnull(xrefs):
        IDs = IDs.split(";")
        xrefs = xrefs.split(";")
        chebi = find_chebi(xrefs)

        for ID in IDs:
            df_mets = df_mets.append({"ID" : ID, "CHEBI" : chebi}, ignore_index = True)

        
df_mets.drop_duplicates(inplace = True)
df_mets

,ID,CHEBI
0,s_0793,chebi:15378
1,s_0794,chebi:15378
2,s_0795,chebi:15378
3,s_0796,chebi:15378
4,s_0797,chebi:15378
...,...,...
2286,s_2976,chebi:15377
2287,s_2994,chebi:15377
2288,s_3226,chebi:15377
2289,s_3449,chebi:15377


### (b) SMILES:

In [8]:
from bioservices import *
import numpy as np

ch = ChEBI()

df_mets["SMILES"] = np.nan
df_mets["name"] = np.nan
df_mets["InChI"] = np.nan

for ind in df_mets.index:
    print(ind)
    chebi_id = df_mets["CHEBI"][ind]
    if not chebi_id is None:
        res = ch.getCompleteEntity(chebi_id.replace("chebi", "CHEBI"))
        try:
            df_mets["name"][ind] = res.chebiAsciiName
        except AttributeError:
            pass
        try:
            df_mets["SMILES"][ind] = res.smiles
        except AttributeError:
            pass
        try:
            df_mets["InChI"][ind] = res.inchi
        except AttributeError:
            pass

0


C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\alexk\anaconda3\envs\Predicting_Km\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


#### Search in PubChem if SMILES and InChI not available yet:

In [11]:
for ind in df_mets.index:

    if pd.isnull(df_mets["SMILES"][ind]) and pd.isnull(df_mets["InChI"][ind]):
        if not pd.isnull(df_mets["name"][ind]):
            cs = get_compounds(df_mets["name"][ind], 'name')
            if len(cs) > 0:
                c = cs[0]
                df_mets["InChI"][ind] = c.inchi
                print(c.synonyms[0])
df_mets.to_pickle(join("..", "..", "data", "DLKcat", "df_metabolites_" +strain + ".pkl"))   

alpha-D-Man-(1->3)-[alpha-D-Man-(1->6)]-beta-D-Man-(1->4)-beta-D-GlcNAc-(1->4)-alpha-D-GlcNAc-diphosphodolichol
cis-3-Chloroacrylic acid
alpha-D-mannosyl-beta-D-mannosyldiacetylchitobiosyldiphosphodolichol
Man-beta1->4GlcNAc-beta1->4GlcNAc-PP-Dol
Man-beta1->4GlcNAc-beta1->4GlcNAc-PP-Dol
CHEBI:132515
dolichyl diphosphate
N-Acetyl-D-glucosaminyldiphosphodolichol
heme O
dolichyl beta-D-glucosyl phosphate
D-Arabino-Hex-2-ulo-Pyranose
D-Arabino-Hex-2-ulo-Pyranose
D-Arabino-Hex-2-ulo-Pyranose
L-Sor
L-Sor
56-73-5
56-73-5
Dolichyl phosphate
Dolichyl phosphate
D-Man
D-Man
D-Man
Mannan
Mannan
Chitosan
D-Rib
D-Rib
Dolichyl D-mannosyl phosphate
beta-D-glucose
beta-D-glucose
beta-D-glucose
beta-D-glucose
D-Gal
D-Gal
D-Gal
mannose-6-phosphate
mannose-6-phosphate
mannose-6-phosphate
Dextrin
Heme-a
Heme-a
GLYCOGEN
GLYCOGEN
CHEBI:57269
trisulfur
5'-Thymidylic acid
5'-Thymidylic acid


In [12]:
df_mets = pd.read_pickle(join("..", "..", "data", "DLKcat", "df_metabolites_" +strain + ".pkl"))   

In [13]:
for ind in df_mets.index:
    if pd.isnull(df_mets["SMILES"][ind]):
        df_mets["SMILES"][ind] = df_mets["InChI"][ind]

In [14]:
def remove_brackets(met):
    return(met.replace("[c]", "").replace("[p]", "").replace("[m]", "").replace("[p]", "").replace("[ce]", "").replace("[n]", "").replace("[e]", ""))

In [17]:
df_reactions = pd.DataFrame(columns = ["reaction ID", "substrate IDs", "product IDs",
                                      "substrate SMILES", "product SMILES"])

reactions = model.reactions

for reaction in reactions:    
    try:    
        product_ids, substrate_ids  = [], []
        product_smiles, substrate_smiles = [], []

        products, substrates = reaction.products, reaction.reactants

        for product in products:
            ID = remove_brackets(met = product.id)
            product_ids.append(ID)
            product_smiles.append(list(df_mets["SMILES"].loc[df_mets["ID"] == ID])[0])

        for substrate in substrates:
            ID = remove_brackets(met = substrate.id)
            substrate_ids.append(ID)
            substrate_smiles.append(list(df_mets["SMILES"].loc[df_mets["ID"] == ID])[0])

        df_reactions = df_reactions.append({"reaction ID" : reaction.id,
                                            "substrate IDs" : substrate_ids, "product IDs" : product_ids,
                                          "substrate SMILES" : substrate_smiles, "product SMILES" : product_smiles},
                                          ignore_index=True)   
    except IndexError:
        print(reaction.id)
        df_reactions = df_reactions.append({"reaction ID" : reaction.id,
                            "substrate IDs" : substrate_ids, "product IDs" : product_ids,
                          "substrate SMILES" : np.nan, "product SMILES" : np.nan},
                          ignore_index=True)   

r_0123
r_0126
r_0127
r_0128
r_0129
r_0130
r_0131
r_0132
r_0133
r_0134
r_0135
r_0137
r_0192
r_0193
r_0244
r_0259
r_0260
r_0261
r_0262
r_0263
r_0264
r_0265
r_0266
r_0267
r_0268
r_0269
r_0270
r_0281
r_0282
r_0283
r_0284
r_0285
r_0286
r_0287
r_0288
r_0289
r_0290
r_0291
r_0292
r_0293
r_0294
r_0295
r_0296
r_0297
r_0298
r_0299
r_0340
r_0341
r_0342
r_0343
r_0361
r_0362
r_0464
r_0518
r_0519
r_0520
r_0521
r_0522
r_0523
r_0596
r_0597
r_0598
r_0599
r_0600
r_0601
r_0602
r_0603
r_0604
r_0605
r_0606
r_0607
r_0608
r_0609
r_0610
r_0611
r_0612
r_0613
r_0614
r_0615
r_0646
r_0647
r_0648
r_0649
r_0650
r_0651
r_0652
r_0653
r_0654
r_0655
r_0791
r_0810
r_0919
r_0920
r_0921
r_0922
r_0993
r_1003
r_1004
r_1005
r_1006
r_1007
r_1010
r_1011
r_1092
r_1094
r_2156
r_2157
r_2158
r_2159
r_2160
r_2168
r_2169
r_2170
r_2171
r_2172
r_2173
r_2174
r_2175
r_2176
r_2177
r_2178
r_2179
r_2180
r_2181
r_2182
r_2183
r_2194
r_2195
r_2196
r_2197
r_2198
r_2199
r_2200
r_2201
r_2202
r_2203
r_2204
r_2205
r_2213
r_2214
r_2215
r_2216
r_2217

### (c) Mapping substrate infos to df_kcat:

In [18]:
ind = 0

df_kcat["substrate SMILES"], df_kcat["product SMILES"] = "" , ""

for ind in df_kcat.index:

    r_ID = df_kcat["# rxnID"][ind]
    if "rvs" in r_ID:
        backward = True
    else:
        backward = False

    end = r_ID[2:].find("_")
    if end != -1:
        r_ID = r_ID[:end+2]

    help_df = df_reactions.loc[df_reactions["reaction ID"] == r_ID]

    if backward:
        df_kcat["substrate SMILES"][ind] = list(help_df["product SMILES"])[0]
        df_kcat["product SMILES"][ind] = list(help_df["substrate SMILES"])[0]
    else:
        df_kcat["substrate SMILES"][ind] = list(help_df["substrate SMILES"])[0]
        df_kcat["product SMILES"][ind] = list(help_df["product SMILES"])[0]
df_kcat

,# rxnID,MNXID,met_model,met_standard_name,Smiles,genes,refID,Kcat value (substrate first),substrate SMILES,product SMILES
0,r_0001_1,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_1887;Kluyveromyces_la...,chebi:16004;chebi:15991,"0.4359,21.0893;","[C[C@@H](O)C([O-])=O, nan]","[nan, CC(=O)C([O-])=O]"
1,r_0001_2,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,2.9483;","[C[C@@H](O)C([O-])=O, nan]","[nan, CC(=O)C([O-])=O]"
2,r_0001_3,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,3.3644;","[C[C@@H](O)C([O-])=O, nan]","[nan, CC(=O)C([O-])=O]"
3,r_0001_4,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,2.0915;","[C[C@@H](O)C([O-])=O, nan]","[nan, CC(=O)C([O-])=O]"
4,r_0002,MNXM731974;MNXM731834,ferricytochrome c;(R)-lactate,ferricytochrome c;(R)-lactate,;C[C@@H](O)C([O-])=O,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:15991;chebi:16004,";21.0893,2.9483","[C[C@@H](O)C([O-])=O, nan]","[nan, CC(=O)C([O-])=O]"
...,...,...,...,...,...,...,...,...,...,...
4639,p_5228_rvs,MNXM729613,D-fructose,D-fructose,OC[C@H]1OC(O)(CO)[C@@H](O)[C@@H]1O,Kluyveromyces_lactis@Seq_1175,chebi:37721,0.4837,[InChI=1S/C6H12O6/c7-2-6(11)5(10)4(9)3(8)1-12-...,[InChI=1S/C6H12O6/c7-2-6(11)5(10)4(9)3(8)1-12-...
4640,r_0486_1_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_2416,chebi:57540;chebi:43474;chebi:17138,12.1500;28.4597;54.7794,"[[H]C(=O)C(O)COP(O)(O)=O, NC(=O)c1ccc[n+](c1)[...",[O[C@H](COP([O-])([O-])=O)C(=O)OP([O-])([O-])=...
4641,r_0486_1_rvs,MNXM10;MNXM733151,"NADH;1,3-bisphospho-D-glycerate",NADH;(2R)-3-phospho-glyceroyl phosphate,NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP([O-])(=O)O...,Kluyveromyces_lactis@Seq_2416,chebi:57945;chebi:57604,31.1839;27.5817,[O[C@H](COP([O-])([O-])=O)C(=O)OP([O-])([O-])=...,"[[H]C(=O)C(O)COP(O)(O)=O, NC(=O)c1ccc[n+](c1)[..."
4642,r_0486_2_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_3619,chebi:57540;chebi:43474;chebi:17138,12.2171;17.5832;35.2701,"[[H]C(=O)C(O)COP(O)(O)=O, NC(=O)c1ccc[n+](c1)[...",[O[C@H](COP([O-])([O-])=O)C(=O)OP([O-])([O-])=...


## 2. Mapping the amino acid sequences to the entries:

Loading amino acid sequences:

In [21]:

fasta = join("..", "..", "data", "DLKcat", "Proteinfasta", strain + ".fasta")
# read FASTA file

df_seq = pd.DataFrame(columns = ["ID", "Sequence"])

for seq_record in SeqIO.parse(open(fasta, mode='r'), 'fasta'):    
    df_seq = df_seq.append({"ID" : seq_record.id, "Sequence" : str(seq_record.seq)}, ignore_index = True)

In [22]:
for ind in df_seq.index:
    df_seq["ID"][ind] = df_seq["ID"][ind].split("@")[1]

In [23]:
df_kcat["Sequences"] = ""

for ind in df_kcat.index:
    sequences = []
    genes = df_kcat["genes"][ind].split(";")
    genes = [gene.split("@")[1] for gene in genes]
    for gene in genes:
        sequences.append(list(df_seq["Sequence"].loc[df_seq["ID"] == gene])[0])

    df_kcat["Sequences"][ind] = sequences

## 3. Calculating reaction fingerprints:

In [24]:
mol_folder = "C:\\Users\\alexk\\substrateprediction-main\\data\\mol-files"
def get_reaction_site_smarts(metabolites):
    reaction_site = ""
    for met in metabolites:
        
        if met[0] == "I":
            mol = Chem.inchi.MolFromInchi(met)
            if mol is not None:
                Smarts = Chem.MolToSmarts(mol)
                
        else:
            mol = Chem.MolFromSmiles(met)
            if mol is not None:
                Smarts = Chem.MolToSmarts(mol)
        reaction_site = reaction_site + "." + Smarts
    return(reaction_site[1:])

def convert_fp_to_array(difference_fp_dict):
    fp = np.zeros(2048)
    for key in difference_fp_dict.keys():
        fp[key] = difference_fp_dict[key]
    return(fp)

In [25]:
df_kcat["structural_fp"], df_kcat["difference_fp"] = "", ""

for ind in df_kcat.index:
    substrates = df_kcat["substrate SMILES"][ind]
    products = df_kcat["product SMILES"][ind]
    
    
    try:
        substrates = list(np.array(substrates)[~pd.isnull(substrates)])
        products = list(np.array(products)[~pd.isnull(products)])
        if len(substrates) > 0 and len(products) > 0:    
            left_site = get_reaction_site_smarts(substrates)
            right_site = get_reaction_site_smarts(products)
            if not pd.isnull(left_site) and not pd.isnull(right_site):

                rxn_forward = AllChem.ReactionFromSmarts(left_site + ">>" + right_site)

                difference_fp = Chem.rdChemReactions.CreateDifferenceFingerprintForReaction(rxn_forward)
                difference_fp =convert_fp_to_array(difference_fp.GetNonzeroElements())
                structural_fp = Chem.rdChemReactions.CreateStructuralFingerprintForReaction(rxn_forward).ToBitString()

                df_kcat["structural_fp"][ind] = structural_fp
                df_kcat["difference_fp"][ind] = difference_fp
    except IndexError:
        pass

### Creating new DataFrame for predictions

In [26]:
df_pred = pd.DataFrame(columns = ["Reaction ID", "Sequence", "difference_fp"])

for ind in df_kcat.index:
    sequences = df_kcat["Sequences"][ind]
    r_ID = df_kcat["# rxnID"][ind]
    diff_fp = df_kcat["difference_fp"][ind]
    
    for seq in sequences:
        df_pred = df_pred.append({"Reaction ID" : r_ID, "Sequence" : seq, "difference_fp" : diff_fp},
                                ignore_index = True)

Calculating ESM1b vectors for all sequences: 

In [27]:
ofile = open(join("..", "..", "data", "DLKcat", "all_sequences_DLKcat" + strain + ".fasta"), "w")
for ind in df_pred.index:
    seq = df_pred["Sequence"][ind]
    if not pd.isnull(seq):
        ofile.write(">" + str(ind) + "\n" + seq[:1020]  + "\n")
ofile.close()

In [29]:
import torch
rep_dict = torch.load(join("..", "..", "data", "DLKcat", "all_sequences_DLKcat" + strain + ".pt"))

df_pred["ESM1b"] = ""

for ind in df_pred.index:
    if not pd.isnull(df_pred["Sequence"][ind]):        
        df_pred["ESM1b"][ind] = rep_dict[str(ind)+".pt"]
        
df_pred.to_pickle(join("..", "..", "data", "DLKcat", "df_pred_" + strain + ".pkl"))

## 4. Making predictions for all entries:

In [30]:
bst = pickle.load(open(join("..", "..", "data", "training_results", "saved_models", 
                            "xgboost_train_and_test.pkl"), "rb"))

bst2 = pickle.load(open(join("..", "..", "data", "training_results", "saved_models", 
                            "xgboost_sequence_only_train_and_test.pkl"), "rb"))

In [32]:
df_pred = pd.read_pickle(join("..", "..", "data", "DLKcat",  "df_pred_" + strain + ".pkl"))

df_pred["log10_kcat_pred"] = np.nan


for ind in df_pred.index:
    diff_fp, esm1b = df_pred["difference_fp"][ind], df_pred["ESM1b"][ind]
    if diff_fp != "":
        X = np.concatenate([diff_fp, esm1b])
        dX = xgb.DMatrix(np.reshape(X, (1,-1)))
        df_pred["log10_kcat_pred"][ind] = bst.predict(dX) 

    else: 
        X = np.array(esm1b)
        dX = xgb.DMatrix(np.reshape(X, (1,-1)))
        df_pred["log10_kcat_pred"][ind] = bst2.predict(dX) 

In [33]:
df_pred["kcat_pred"] = 10**df_pred["log10_kcat_pred"]
df_kcat["new kcat"] = ""

for ind in df_kcat.index:
    r_ID = df_kcat["# rxnID"][ind]

    new_kcats = list(df_pred["kcat_pred"].loc[df_pred["Reaction ID"] == r_ID])
    kcat_string = ""
    for kcat in new_kcats:
        kcat_string = kcat_string + "," + str(kcat)
    df_kcat["new kcat"][ind] = kcat_string[1:]

In [34]:
df = df_kcat.copy()
df.drop(columns = ["Sequences", "structural_fp", "difference_fp", "substrate SMILES", "product SMILES"], inplace = True)
df

,# rxnID,MNXID,met_model,met_standard_name,Smiles,genes,refID,Kcat value (substrate first),new kcat
0,r_0001_1,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_1887;Kluyveromyces_la...,chebi:16004;chebi:15991,"0.4359,21.0893;","42.50704199647334,34.73993033852424"
1,r_0001_2,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,2.9483;","34.73993033852424,29.115880267552587"
2,r_0001_3,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,3.3644;","34.73993033852424,39.74852411815562"
3,r_0001_4,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"21.0893,2.0915;","34.73993033852424,19.430814721386742"
4,r_0002,MNXM731974;MNXM731834,ferricytochrome c;(R)-lactate,ferricytochrome c;(R)-lactate,;C[C@@H](O)C([O-])=O,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:15991;chebi:16004,";21.0893,2.9483","34.73993033852424,29.115880267552587"
...,...,...,...,...,...,...,...,...,...
4639,p_5228_rvs,MNXM729613,D-fructose,D-fructose,OC[C@H]1OC(O)(CO)[C@@H](O)[C@@H]1O,Kluyveromyces_lactis@Seq_1175,chebi:37721,0.4837,48.86895265580309
4640,r_0486_1_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_2416,chebi:57540;chebi:43474;chebi:17138,12.1500;28.4597;54.7794,77.89304558174084
4641,r_0486_1_rvs,MNXM10;MNXM733151,"NADH;1,3-bisphospho-D-glycerate",NADH;(2R)-3-phospho-glyceroyl phosphate,NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP([O-])(=O)O...,Kluyveromyces_lactis@Seq_2416,chebi:57945;chebi:57604,31.1839;27.5817,68.63817583581475
4642,r_0486_2_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_3619,chebi:57540;chebi:43474;chebi:17138,12.2171;17.5832;35.2701,77.66200561087881


In [35]:
def create_kcat_array(old_kcat, new_kcat):
    new_kcat_array = []
    for kcat in old_kcat:
        if kcat == "":
            new_kcat_array.append("")
        else:
            if len(kcat.split(",")) == len(new_kcat.split(",")):
                new_kcat_array.append(new_kcat)
            else: 
                print(ind, new_kcat, old_kcat)
                
    return(new_kcat_array)

def create_kcat_string(kcat_array):
    if len(kcat_array) == 1:
        return(kcat_array[0])
    else:
        kcat_string = ""
        for i, kcat in enumerate(kcat_array):
            kcat_string = kcat_string + kcat + ";"
    return(kcat_string[:-1])

In [36]:
for ind in df.index:
    if not pd.isnull(df["Kcat value (substrate first)"][ind]):
        new_kcat = df["new kcat"][ind]
        old_kcat = df["Kcat value (substrate first)"][ind].split(";")

        kcat_array= create_kcat_array(old_kcat, new_kcat)
        kcat_string = create_kcat_string(kcat_array)
        df["Kcat value (substrate first)"][ind] = kcat_string
        
df.drop(columns = ["new kcat"], inplace = True)
df

,# rxnID,MNXID,met_model,met_standard_name,Smiles,genes,refID,Kcat value (substrate first)
0,r_0001_1,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_1887;Kluyveromyces_la...,chebi:16004;chebi:15991,"42.50704199647334,34.73993033852424;"
1,r_0001_2,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"34.73993033852424,29.115880267552587;"
2,r_0001_3,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"34.73993033852424,39.74852411815562;"
3,r_0001_4,MNXM731834;MNXM731974,(R)-lactate;ferricytochrome c,(R)-lactate;ferricytochrome c,C[C@@H](O)C([O-])=O;,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:16004;chebi:15991,"34.73993033852424,19.430814721386742;"
4,r_0002,MNXM731974;MNXM731834,ferricytochrome c;(R)-lactate,ferricytochrome c;(R)-lactate,;C[C@@H](O)C([O-])=O,Kluyveromyces_lactis@Seq_2031;Kluyveromyces_la...,chebi:15991;chebi:16004,";34.73993033852424,29.115880267552587"
...,...,...,...,...,...,...,...,...
4639,p_5228_rvs,MNXM729613,D-fructose,D-fructose,OC[C@H]1OC(O)(CO)[C@@H](O)[C@@H]1O,Kluyveromyces_lactis@Seq_1175,chebi:37721,48.86895265580309
4640,r_0486_1_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_2416,chebi:57540;chebi:43474;chebi:17138,77.89304558174084;77.89304558174084;77.8930455...
4641,r_0486_1_rvs,MNXM10;MNXM733151,"NADH;1,3-bisphospho-D-glycerate",NADH;(2R)-3-phospho-glyceroyl phosphate,NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP([O-])(=O)O...,Kluyveromyces_lactis@Seq_2416,chebi:57945;chebi:57604,68.63817583581475;68.63817583581475
4642,r_0486_2_fwd,MNXM8;MNXM9;MNXM739945,NAD;phosphate;glyceraldehyde 3-phosphate,NAD(+);phosphate;glyceraldehyde 3-phosphate,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,Kluyveromyces_lactis@Seq_3619,chebi:57540;chebi:43474;chebi:17138,77.66200561087881;77.66200561087881;77.6620056...


In [38]:
df.to_csv(join("..", "..", "data", "DLKcat", "PredcitedKcat343species", 
                 strain + "_PredictionResults_V2.txt"), sep = "\t", index = False)